### Type of langchain chat memory buffers
- ConversationBufferMemory, save previous conversations to chat history
- ConversationSummaryMemory, summarize the previous conversations
- ConversationBufferWindowMemory, fix window size
- ConversationKGMemory, knowledge graph

### Build a chatbot using langchain:

In [ ]:
# ! pip install -U langchain

In [1]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from getpass import getpass

In [2]:
OPENAI_API_KEY = getpass()

In [3]:
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, 
    temperature=0,
    model="gpt-4"
    )

Manual prompt with LLMChain

In [4]:
from langchain import PromptTemplate

template = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

{history}
Human: {human_input}
Assistant:"""

prompt = PromptTemplate(input_variables=['history', 'human_input'], template=template)


In [5]:
chain = LLMChain(llm=llm, prompt=prompt)

output = chain.run(human_input="what is quantum physics? Answer in under 50 characters. sign off your message with -Dr.roboto", history="")
print(output)

Quantum physics studies particles at the smallest scales. -Dr.roboto


ConversationChain with predefined prompt

In [6]:
from langchain import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY, 
    temperature=0,
    model="gpt-3.5-turbo"
    )

conversation = ConversationChain(llm=llm)

In [7]:
conversation.prompt.template

'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'

In [8]:
conversation("what is quantum physics? Answer in under 50 characters. sign off your message with -Dr.roboto")

{'input': 'what is quantum physics? Answer in under 50 characters. sign off your message with -Dr.roboto',
 'history': '',
 'response': 'The study of subatomic particles and their interactions. -Dr.roboto'}

In [9]:
conversation_with_buffer = ConversationChain(llm=llm, memory=ConversationBufferMemory())

In [10]:
conversation_with_buffer.prompt.template

'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'

In [11]:
from langchain.callbacks import get_openai_callback
def chat_and_count_tokens(chain, query: str) -> int:
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f"Spent a total of {cb.total_tokens} tokens.")
    
    return result

In [12]:
chat_and_count_tokens(conversation_with_buffer, "I want to explore the potential of integrating large language models with external knowledge.")

Spent a total of 376 tokens.


"That's a great topic! Integrating large language models with external knowledge can have numerous benefits. By combining the power of language models with external knowledge sources, you can enhance the model's understanding and generate more accurate and contextually relevant responses.\n\nThere are several approaches you can take to integrate external knowledge with large language models. One common method is to use knowledge graphs, which represent information in a structured format. By connecting the language model to a knowledge graph, you can enable it to access and reason over a vast amount of factual information.\n\nAnother approach is to leverage pre-trained models that have been specifically trained on domain-specific knowledge. For example, you can fine-tune a language model on a dataset that contains information from a specific domain, such as medicine or law. This allows the model to have a deeper understanding of the domain-specific concepts and terminology.\n\nAdditiona

In [13]:
chat_and_count_tokens(conversation_with_buffer, "Which way is better?")

Spent a total of 657 tokens.


'The choice of which approach is better for integrating external knowledge with large language models depends on the specific use case and requirements. Each approach has its own advantages and considerations.\n\nUsing knowledge graphs can provide a structured representation of information, allowing for efficient querying and reasoning. This approach is particularly useful when dealing with factual information that can be organized into a graph-like structure. Knowledge graphs can also capture relationships between entities, enabling the language model to make more nuanced and contextually relevant responses.\n\nOn the other hand, fine-tuning language models on domain-specific datasets can be beneficial when you need the model to have a deeper understanding of a specific domain. This approach allows the model to learn domain-specific concepts, terminology, and patterns, which can improve its performance in that particular domain. However, fine-tuning requires access to domain-specific 

In [14]:
chat_and_count_tokens(conversation_with_buffer, "What is a knowledge graph?")

Spent a total of 902 tokens.


'A knowledge graph is a structured representation of information that captures relationships between entities. It is typically represented as a graph, where nodes represent entities and edges represent relationships between those entities. Each node and edge in the graph can have attributes that provide additional information.\n\nKnowledge graphs are designed to organize and represent factual information in a way that is easily queryable and computationally efficient. They can be used to store and reason over a wide range of knowledge, such as facts, concepts, and relationships between entities.\n\nFor example, a knowledge graph could represent information about movies, actors, and directors. Each movie, actor, and director would be represented as a node in the graph, and the relationships between them (e.g., "directed by," "acted in") would be represented as edges. The nodes and edges can also have attributes, such as the release year of a movie or the birth date of an actor.\n\nKnowl

In [15]:
chat_and_count_tokens(conversation_with_buffer, "Can you show me an example of knowledge graph?")

Spent a total of 1295 tokens.


'Sure! Here\'s an example of a simple knowledge graph representing information about movies, actors, and directors:\n\nNodes:\n- Movie: "The Shawshank Redemption", "Inception", "The Dark Knight"\n- Actor: "Morgan Freeman", "Leonardo DiCaprio", "Christian Bale"\n- Director: "Frank Darabont", "Christopher Nolan"\n\nEdges:\n- "directed by": connects the Director nodes to the Movie nodes\n- "acted in": connects the Actor nodes to the Movie nodes\n\nAttributes:\n- Movie nodes can have attributes like "release year" and "genre"\n- Actor nodes can have attributes like "birth date" and "nationality"\n- Director nodes can have attributes like "birth date" and "nationality"\n\nHere\'s how the knowledge graph would look like:\n\n```\n(Movie) The Shawshank Redemption --[directed by]--> (Director) Frank Darabont\n(Movie) The Shawshank Redemption --[acted in]--> (Actor) Morgan Freeman\n\n(Movie) Inception --[directed by]--> (Director) Christopher Nolan\n(Movie) Inception --[acted in]--> (Actor) Leon

In [16]:
print(conversation_with_buffer.memory.buffer)

Human: I want to explore the potential of integrating large language models with external knowledge.
AI: That's a great topic! Integrating large language models with external knowledge can have numerous benefits. By combining the power of language models with external knowledge sources, you can enhance the model's understanding and generate more accurate and contextually relevant responses.

There are several approaches you can take to integrate external knowledge with large language models. One common method is to use knowledge graphs, which represent information in a structured format. By connecting the language model to a knowledge graph, you can enable it to access and reason over a vast amount of factual information.

Another approach is to leverage pre-trained models that have been specifically trained on domain-specific knowledge. For example, you can fine-tune a language model on a dataset that contains information from a specific domain, such as medicine or law. This allows th

ConversationKGMemory

In [19]:
# !pip install -qU networkx


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [33]:
from langchain.chains.conversation.memory import ConversationKGMemory

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""

prompt = PromptTemplate(input_variables=['history', 'input'], template=template)

conversation_kg_chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=ConversationKGMemory(llm=llm),
    verbose=True
)

In [34]:
chat_and_count_tokens(conversation_kg_chain, "Hi, What's up?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Hi, What's up?
AI:

> Finished chain.
Spent a total of 1043 tokens.


"Hello! I'm an AI and I'm here to chat with you. How can I assist you today?"

In [35]:
chat_and_count_tokens(conversation_kg_chain, "My name is Kelvin and I'm helping Will. He's an engineer.")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: My name is KElvin and I'm helping Will. He's an engineer.
AI:

> Finished chain.
Spent a total of 1166 tokens.


"Hello Kelvin! It's nice to meet you. How can I assist you with helping Will?"

In [36]:
chat_and_count_tokens(conversation_kg_chain, "What do you know about will?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

On Will: Will is an engineer.

Conversation:
Human: What do you know about will?
AI:

> Finished chain.
Spent a total of 1140 tokens.


'Will is an engineer.'

In [37]:
conversation_kg_chain.memory.kg.get_triples()

[('Kelvin', 'Will', 'is helping'), ('Will', 'engineer', 'is an')]

### ConversationSummary

In [47]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversation_sum_chain = ConversationChain(
    llm=llm, 
    memory=ConversationSummaryMemory(llm=llm),
    verbose=True
)

In [48]:
chat_and_count_tokens(conversation_sum_chain, "Hi, What's up?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, What's up?
AI:

> Finished chain.
Spent a total of 323 tokens.


"Hello! I'm an AI designed to assist with various tasks and provide information. Right now, I'm here to chat with you. How can I help you today?"

In [49]:
chat_and_count_tokens(conversation_sum_chain, "My name is Kelvin and I'm helping Will. He's an engineer.")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human greets the AI and asks what it is doing. The AI explains that it is an AI designed to assist with tasks and provide information, and it is currently available to chat and help the human.
Human: My name is Kelvin and I'm helping Will. He's an engineer.
AI:

> Finished chain.
Spent a total of 433 tokens.


"Hello Kelvin! It's nice to meet you. How can I assist you with helping Will, the engineer?"

In [50]:
chat_and_count_tokens(conversation_sum_chain, "What do you know about will?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human greets the AI and asks what it is doing. The AI explains that it is an AI designed to assist with tasks and provide information, and it is currently available to chat and help the human. The human introduces themselves as Kelvin and mentions that they are helping Will, an engineer. The AI greets Kelvin and asks how it can assist with helping Will.
Human: What do you know about will?
AI:

> Finished chain.
Spent a total of 598 tokens.


"Will is an engineer who you mentioned you are helping. As an AI, I don't have personal knowledge about individuals unless it has been shared with me in our conversation. However, I can assist you with general information about engineering or any specific questions you may have about Will's work or field of expertise."